# Steam Recommendations Project

Project to build a recommendation engine for steam games.

### Retrieve data

Use Steam Web API to retrieve public user data.

In [1]:
#import libraries
import random as rnd
import numpy as np
import pandas as pd
import requests

rnd.seed(8524)

In [92]:
# get all steam app ids
response = requests.get("https://api.steampowered.com/ISteamApps/GetAppList/v2/")

steam_apps=pd.DataFrame(response.json()["applist"]["apps"])

In [ ]:
# get app general info
# todo

In [91]:
# get app review info

parameters = {'json': 1, 
              'filter': 'recent', 
              'language': 'all',
              'cursor': '*',
              'review_type': 'all',
              'purchase_type': 'all',
              'num_per_page': '100',
              'filter_offtopic_activity': 0
              }

review_summaries = list()

def prep_reviews_df(resp,appid):
    # resp is a dict of api response

    author_df = pd.DataFrame([item["author"] for item in resp])

    reviews_df=pd.DataFrame(resp)
    reviews_df=reviews_df.drop(columns=['author'])
    reviews_df=reviews_df.join(author_df)

    reviews_df["appid"]=[appid]*len(reviews_df)

    return reviews_df

for this_app in steam_apps["appid"]: # for testing: ['2717080']

    while 1:
        # get document from API (API only allows retrieval of 20 reviews at a time)
        response = requests.get("https://store.steampowered.com/appreviews/" + str(this_app), params=parameters)
        response = response.json() # convert to dict

        if response["query_summary"]["num_reviews"] == 0:
            break # exit when no more reviews to retrieve for title

        # update reviews dfs
        if parameters['cursor'] == '*': # first iteration
            review_summaries = pd.DataFrame(response["query_summary"],index=[0])
            review_summaries=review_summaries.drop(columns=['num_reviews'])
            review_summaries["appid"] = this_app

            reviews_all = prep_reviews_df(response["reviews"],this_app)
        else:
            reviews_all = pd.concat([reviews_all, prep_reviews_df(response["reviews"],this_app)])

        # update cursor for next API request
        parameters["cursor"] = response["cursor"].encode()

In [ ]:
# load steam api key
with open("steam_api_key.txt", 'r') as file:
    steam_key = file.read()

# get id list
id_list = reviews_all["steamid"].unique()

In [118]:
# get user summaries (only those who left at least one review for one app) - this is not really useful

# load steam api key
with open("steam_api_key.txt", 'r') as file:
    steam_key = file.read()

id_list = reviews_all["steamid"].unique()

for x in np.arange(0,len(id_list),100): # can only request 100 ids at a time
    
    # get ids
    if x+100 < len(id_list):
        id_request = id_list[x:x+100]
    else:
        id_request = id_list[x:]

    id_request_str = ",".join(id_request) # format id list for api call

    parameters = {"key": steam_key, "steamids": id_request_str}

    response = requests.get("http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002",params=parameters)
    response = response.json()

    # update reviews dfs
    if x == 0: # first iteration
        users_all = pd.DataFrame(response["response"]["players"])
    else:
        users_all = pd.concat([users_all, pd.DataFrame(response["response"]["players"])])

users_all.reset_index(drop=True,inplace=True)

Note for generating random ids (see steam doc):

        id = rnd.randrange(1,1000000)
        id64 = id*2 + 76561197960265728 + 1

In [125]:
# get user games owned
# todo testing

for id in ['76561197960435530']:  #id_list:

    parameters = {"key": steam_key, "steamid": id}

    response = requests.get("http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001",params=parameters)
    response = response.json()

    # update reviews dfs
    if id == id_list[0]: # first iteration
        users_all = pd.DataFrame(response["response"]["players"])
    else:
        users_all = pd.concat([users_all, pd.DataFrame(response["response"]["players"])])

users_all.reset_index(drop=True,inplace=True)

KeyError: 'players'